In [21]:
import json
import cv2
import os
import numpy as np

In [22]:
def display(*img):
    for i,im in enumerate(img):
        cv2.imshow(str(i),im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [23]:
def get_images():
    images=os.listdir('./main_data/main_images')
    temp={}
    for i in images:
        temp[i]=cv2.imread(f'./main_data/main_images/{i}').copy()
    return temp

In [24]:
data=open('./main_data/main_json.json').read()
data=json.loads(data)
type(data)

dict

In [25]:
filter=['filename','x','y','width','height','0','1']
def get_depth(d):

    o = []
    def v_check(key,val):

        if key in filter or val in filter :
            o.append(str(f'{key}:{val}'))

    def check(d):
        if isinstance(d, dict):

            for k,v in d.items():

                if isinstance(v, (dict, list)):
                    check(v)

                else:
                    if v:
                       v_check(k,v)
                   

        elif isinstance(d, (list)):
            for v in d:

                if isinstance(v, (dict, list)):

                    check(v)

                else:
                    pass
                    # v_check(v)

    check(d)
    return o

data=get_depth(data)
# data

In [26]:
def rotation(img,angle=0,scale=1):
    center=img.shape[1]//2,img.shape[0]//2
    M=cv2.getRotationMatrix2D(center,angle,scale)
    rotated_img=cv2.warpAffine(img,M,(img.shape[1],img.shape[0]))
    return rotated_img

In [57]:
save_dir='aug_images'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
def rotate(filename,image,points):
    x,y,w,h=[int(x) for x in points]
    cv2.rectangle(image,(x,y),(x+w,y+h),255,3)
    display(image)
    return image
    # cv2.imwrite(f'{save_dir}/{filename}.jpg',image)


In [58]:
images=get_images()
image=''
mask_image=''
points=[]
for i in data:
    # print(i)
    if i.split(':')[0] == 'filename':
        image=images[i.split(':')[1]]
        filename=i.split(':')[1].split('.')[0]
        image=rotation(image,45)
    else:
        if i.split(":")[0] in ['x','y','width','height']:
            points.append(i.split(":")[1])
        else:
            image=rotate(filename,image,points)
            points=[]